# Summary
- mincorr and minlinreg are similar. maxcorr and maxlinreg are similar. 
- spearman among different layers is lower (even goes negative) in lower layers for:
    - bert_large
    - openai
    - bert_base
  
  but not true for elmo or calypso

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os.path import basename, dirname
import torch

In [2]:
# Set `res_d`, `network_l`, `num_neurons_d`
base = "/data/sls/temp/johnmwu/contextual-corr-analysis/results7_"
res_fname = {method : base + method for method in 
                {"maxcorr", "mincorr", "maxlinreg", "minlinreg", "cca", "lincka"}}

res_d = {}
res_d["maxcorr"] = json.load(open(res_fname["maxcorr"], "r"))
res_d["mincorr"] = json.load(open(res_fname["mincorr"], "r"))
res_d["maxlinreg"] = json.load(open(res_fname["maxlinreg"], "r"))
res_d["minlinreg"] = json.load(open(res_fname["minlinreg"], "r"))

network_l = [network for network in res_d["maxcorr"]]

num_neurons_d = {}
for network in network_l:
    num_neurons_d[network] = len(res_d["maxcorr"][network])

# MaxCorr, MinCorr, MaxLinReg, MinLinReg

### Sorts

In [3]:
rk_methods = {"maxcorr", "mincorr", "maxlinreg", "minlinreg"}

In [4]:
neuron_sorts = {network: {} for network in network_l}
for network in network_l:
    for method in rk_methods:
        neuron_sorts[network][method] = [neuron[0] for neuron in res_d[method][network]]

In [5]:
# Set `rk_ar_d`, `rk_df_d`, `spearman_d`
rk_ar_d = {}
rk_df_d = {}
spearman_d = {}
for network in network_l:
    # rk_ar[method_ix, neuron] = rank
    # need to invert the permutation
    rk_ar = np.stack([
            [neuron_sorts[network][method].index(neuron) for neuron in range(len(neuron_sorts[network][method]))]
                      for method in rk_methods
    ]).T
    rk_df = pd.DataFrame(rk_ar, columns=list(rk_methods))
    
    rk_ar_d[network] = rk_ar
    rk_df_d[network] = rk_df
    spearman_d[network] = rk_df.corr()

In [6]:
for network in network_l:
    print(network)
    # print(rk_df_d[network])
    print(spearman_d[network])

bert_large_cased_0
            maxcorr  minlinreg  maxlinreg   mincorr
maxcorr    1.000000  -0.177012   0.648555 -0.217173
minlinreg -0.177012   1.000000   0.281875  0.812337
maxlinreg  0.648555   0.281875   1.000000  0.246626
mincorr   -0.217173   0.812337   0.246626  1.000000
bert_large_cased_1
            maxcorr  minlinreg  maxlinreg   mincorr
maxcorr    1.000000   0.253680   0.966645  0.049593
minlinreg  0.253680   1.000000   0.314057  0.695416
maxlinreg  0.966645   0.314057   1.000000  0.107344
mincorr    0.049593   0.695416   0.107344  1.000000
bert_large_cased_2
            maxcorr  minlinreg  maxlinreg   mincorr
maxcorr    1.000000   0.468780   0.977872  0.183748
minlinreg  0.468780   1.000000   0.482464  0.727491
maxlinreg  0.977872   0.482464   1.000000  0.196051
mincorr    0.183748   0.727491   0.196051  1.000000
bert_large_cased_3
            maxcorr  minlinreg  maxlinreg   mincorr
maxcorr    1.000000   0.373058   0.980595  0.119859
minlinreg  0.373058   1.000000   0.39966

### Distributions

In [7]:
for network in network_l:
    print("\n\n" + network + ":")
    for method in rk_methods:
        print("\n" + method)
        corr_ar = np.stack([
            list(notated_neuron[1].values())
            for notated_neuron in res_d[method][network]
        ])
        corr_df = pd.DataFrame(np.abs(corr_ar),
                              columns=[net for net in network_l if net != network])
        
        print("\nmaxcounts")
        print(corr_df.idxmax(axis=1).value_counts())
        
        print("\nmincounts")
        print(corr_df.idxmin(axis=1).value_counts())
        
        print("\ncorr_df_pctl")
        corr_df_pctl = pd.DataFrame()
        corr_df_pctl["max"] = corr_df.max(axis=1)
        corr_df_pctl["min"] = corr_df.min(axis=1)
        corr_df_pctl["med"] = corr_df.median(axis=1)
        print(corr_df_pctl.rank().corr()) # spearman
        
#         corr_df.plot()
#         plt.xlabel("neuron (sorted order)")
#         plt.ylabel("correlation")
#         plt.show()
        
#         plt.hist(corr_ar.ravel(), bins=100)
#         plt.xlabel("correlation")
#         plt.ylabel("count")
#         plt.show()



bert_large_cased_0:

maxcorr

maxcounts
bert_large_cased_1                  981
bert_base_cased_0                    40
calypso_transformer_6_512_base_0      2
openai_transformer_2                  1
dtype: int64

mincounts
elmo_4x4096_512_2                   281
elmo_4x4096_512_3                   111
bert_large_cased_22                  65
bert_large_cased_21                  58
bert_large_cased_23                  46
openai_transformer_12                39
bert_base_cased_10                   39
elmo_4x4096_512_1                    37
calypso_transformer_6_512_base_6     29
elmo_4x4096_512_4                    29
bert_base_cased_11                   25
elmo_original_1                      25
bert_large_cased_18                  23
bert_large_cased_19                  22
elmo_original_2                      20
bert_base_cased_12                   17
bert_base_cased_9                    15
bert_large_cased_17                  12
openai_transformer_11                11
bert_large_cas

          max       min       med
max  1.000000  0.183748  0.314837
min  0.183748  1.000000  0.855940
med  0.314837  0.855940  1.000000

minlinreg

maxcounts
bert_large_cased_3    1024
dtype: int64

mincounts
openai_transformer_12               970
calypso_transformer_6_512_base_0     35
elmo_4x4096_512_0                    14
elmo_original_0                       2
elmo_4x4096_512_4                     1
openai_transformer_11                 1
openai_transformer_0                  1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.482464  0.729536
min  0.482464  1.000000  0.864348
med  0.729536  0.864348  1.000000

maxlinreg

maxcounts
bert_large_cased_3    1024
dtype: int64

mincounts
openai_transformer_12               970
calypso_transformer_6_512_base_0     35
elmo_4x4096_512_0                    14
elmo_original_0                       2
elmo_4x4096_512_4                     1
openai_transformer_11                 1
openai_transformer_0               

openai_transformer_12               978
calypso_transformer_6_512_base_0     33
elmo_4x4096_512_0                     7
elmo_original_0                       4
elmo_4x4096_512_4                     1
openai_transformer_0                  1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.439943  0.731771
min  0.439943  1.000000  0.863522
med  0.731771  0.863522  1.000000

maxlinreg

maxcounts
bert_large_cased_5    1024
dtype: int64

mincounts
openai_transformer_12               978
calypso_transformer_6_512_base_0     33
elmo_4x4096_512_0                     7
elmo_original_0                       4
elmo_4x4096_512_4                     1
openai_transformer_0                  1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.439943  0.731771
min  0.439943  1.000000  0.863522
med  0.731771  0.863522  1.000000

mincorr

maxcounts
bert_large_cased_5    740
bert_large_cased_3    284
dtype: int64

mincounts
elmo_4x4096_512_2        

          max       min       med
max  1.000000  0.565392  0.816925
min  0.565392  1.000000  0.869848
med  0.816925  0.869848  1.000000

mincorr

maxcounts
bert_large_cased_7                  780
bert_large_cased_5                  243
calypso_transformer_6_512_base_0      1
dtype: int64

mincounts
elmo_4x4096_512_2                   202
elmo_4x4096_512_3                   134
elmo_original_2                      63
elmo_original_0                      54
openai_transformer_12                53
elmo_4x4096_512_4                    52
calypso_transformer_6_512_base_6     50
bert_base_cased_12                   48
elmo_4x4096_512_0                    42
elmo_4x4096_512_1                    35
elmo_original_1                      31
openai_transformer_0                 30
bert_large_cased_23                  27
bert_large_cased_24                  25
bert_base_cased_11                   21
bert_base_cased_10                   17
bert_large_cased_21                  14
openai_transformer_1


maxcounts
bert_large_cased_10                 668
bert_large_cased_8                  355
calypso_transformer_6_512_base_0      1
dtype: int64

mincounts
elmo_4x4096_512_2                   200
elmo_4x4096_512_3                   155
openai_transformer_12                75
elmo_original_0                      65
elmo_original_2                      64
elmo_4x4096_512_4                    55
elmo_4x4096_512_0                    46
calypso_transformer_6_512_base_6     41
bert_base_cased_12                   38
openai_transformer_0                 34
elmo_4x4096_512_1                    32
elmo_original_1                      23
openai_transformer_1                 21
calypso_transformer_6_512_base_0     19
openai_transformer_11                16
bert_base_cased_11                   14
bert_large_cased_24                  11
openai_transformer_6                 10
calypso_transformer_6_512_base_5     10
openai_transformer_3                  9
openai_transformer_2                  9
opena

...

          max       min       med
max  1.000000  0.391357  0.870528
min  0.391357  1.000000  0.620319
med  0.870528  0.620319  1.000000

mincorr

maxcounts
calypso_transformer_6_512_base_3    905
calypso_transformer_6_512_base_1    119
dtype: int64

mincounts
bert_large_cased_8                  75
bert_large_cased_23                 70
bert_large_cased_24                 61
elmo_original_0                     61
bert_large_cased_22                 50
bert_large_cased_5                  47
bert_large_cased_19                 43
openai_transformer_12               39
bert_large_cased_21                 37
bert_base_cased_12                  36
bert_large_cased_2                  31
openai_transformer_5                30
elmo_4x4096_512_0                   28
bert_large_cased_7                  28
bert_large_cased_20                 28
elmo_4x4096_512_2                   27
elmo_4x4096_512_3                   25
bert_large_cased_9                  22
bert_large_cased_3                  21

          max       min       med
max  1.000000  0.380591  0.593576
min  0.380591  1.000000  0.811509
med  0.593576  0.811509  1.000000


calypso_transformer_6_512_base_5:

maxcorr

maxcounts
calypso_transformer_6_512_base_4    811
calypso_transformer_6_512_base_6    213
dtype: int64

mincounts
elmo_original_0                     143
elmo_4x4096_512_0                    73
bert_large_cased_22                  63
bert_large_cased_23                  59
bert_large_cased_8                   52
bert_large_cased_5                   50
bert_large_cased_24                  46
bert_large_cased_21                  30
bert_large_cased_2                   29
bert_large_cased_9                   28
bert_base_cased_12                   28
bert_large_cased_6                   27
openai_transformer_5                 26
bert_large_cased_4                   25
bert_large_cased_7                   24
openai_transformer_12                24
openai_transformer_2                 21
bert_large_cased_18     


maxcounts
calypso_transformer_6_512_base_0    200
elmo_original_0                     158
calypso_transformer_6_512_base_1    114
openai_transformer_0                 78
bert_large_cased_20                  68
bert_base_cased_1                    32
bert_large_cased_0                   30
calypso_transformer_6_512_base_2     28
elmo_original_1                      24
elmo_4x4096_512_4                    22
openai_transformer_8                 20
openai_transformer_12                18
bert_base_cased_0                    16
calypso_transformer_6_512_base_4     16
openai_transformer_7                 14
bert_large_cased_1                   14
openai_transformer_1                 14
elmo_4x4096_512_1                    14
calypso_transformer_6_512_base_5     12
elmo_original_2                      12
calypso_transformer_6_512_base_3     10
openai_transformer_5                 10
openai_transformer_3                 10
openai_transformer_10                 8
openai_transformer_11        


maxcounts
elmo_4x4096_512_3    661
elmo_4x4096_512_1    363
dtype: int64

mincounts
openai_transformer_12               511
openai_transformer_0                448
openai_transformer_1                 31
bert_large_cased_24                  15
calypso_transformer_6_512_base_0      8
openai_transformer_5                  4
bert_base_cased_12                    4
elmo_original_0                       3
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.663853  0.874547
min  0.663853  1.000000  0.851224
med  0.874547  0.851224  1.000000

maxlinreg

maxcounts
elmo_4x4096_512_3    661
elmo_4x4096_512_1    363
dtype: int64

mincounts
openai_transformer_12               511
openai_transformer_0                448
openai_transformer_1                 31
bert_large_cased_24                  15
calypso_transformer_6_512_base_0      8
openai_transformer_5                  4
bert_base_cased_12                    4
elmo_original_0                       3
dtype: int64

co


maxcounts
elmo_4x4096_512_3    1024
dtype: int64

mincounts
openai_transformer_0                629
calypso_transformer_6_512_base_0    125
elmo_original_0                     112
openai_transformer_12               109
openai_transformer_1                 19
elmo_4x4096_512_0                    18
bert_base_cased_0                     5
bert_large_cased_24                   3
openai_transformer_5                  3
bert_large_cased_23                   1
dtype: int64

corr_df_pctl
          max       min       med
max  1.000000  0.768030  0.906754
min  0.768030  1.000000  0.899234
med  0.906754  0.899234  1.000000

mincorr

maxcounts
elmo_4x4096_512_3    1024
dtype: int64

mincounts
openai_transformer_0                82
elmo_original_0                     65
bert_large_cased_5                  48
elmo_4x4096_512_0                   47
bert_large_cased_8                  41
bert_large_cased_23                 37
bert_large_cased_22                 34
bert_base_cased_12               

# LinCKA

In [8]:
cka_d = torch.load(res_fname["lincka"])
df = pd.DataFrame(cka_d)

In [9]:
network = "bert_large_cased_0"
df.sort_values(by=network)[network]

calypso_transformer_6_512_base_6    0.333303
calypso_transformer_6_512_base_2    0.338901
calypso_transformer_6_512_base_3    0.361677
calypso_transformer_6_512_base_5    0.364045
bert_large_cased_23                 0.365734
calypso_transformer_6_512_base_4    0.375722
bert_large_cased_24                 0.383112
bert_large_cased_22                 0.392960
bert_large_cased_21                 0.402072
openai_transformer_12               0.407272
calypso_transformer_6_512_base_1    0.411119
openai_transformer_11               0.418603
openai_transformer_10               0.423574
openai_transformer_9                0.423841
bert_large_cased_18                 0.425290
bert_large_cased_20                 0.427148
elmo_4x4096_512_3                   0.431214
bert_base_cased_11                  0.432538
bert_large_cased_19                 0.435088
elmo_4x4096_512_2                   0.437690
openai_transformer_6                0.438417
bert_base_cased_10                  0.440066
openai_tra

# CCA

In [10]:
cca_d = torch.load(res_fname["cca"])

In [11]:
corrs, sv_similarities, pw_similarities = cca_d['corrs'], cca_d['sv_similarities'], cca_d['pw_similarities']

In [12]:
sv_sim_df = pd.DataFrame(sv_similarities)
pw_sim_df = pd.DataFrame(pw_similarities)
corrs_df = pd.DataFrame(corrs)
maxcorrs_df = corrs_df.applymap(lambda t: float('nan') if type(t) is float else t.max().item())

### SVCCA

In [13]:
network = "bert_large_cased_0"
sv_sim_df.sort_values(by=network)[network]

elmo_4x4096_512_4                   0.277943
openai_transformer_12               0.291095
calypso_transformer_6_512_base_6    0.291605
elmo_original_2                     0.291956
bert_large_cased_24                 0.297135
openai_transformer_11               0.305676
bert_base_cased_12                  0.315521
bert_large_cased_23                 0.316317
calypso_transformer_6_512_base_5    0.316620
openai_transformer_10               0.318087
bert_large_cased_22                 0.321090
elmo_4x4096_512_3                   0.321376
bert_large_cased_21                 0.324047
bert_large_cased_20                 0.326096
openai_transformer_9                0.326714
bert_large_cased_19                 0.328506
bert_base_cased_11                  0.329231
bert_large_cased_18                 0.334123
openai_transformer_8                0.336102
bert_base_cased_10                  0.339445
bert_large_cased_17                 0.345119
calypso_transformer_6_512_base_4    0.348826
openai_tra

### PWCCA

In [14]:
network = "bert_large_cased_0"
pw_sim_df.sort_values(by=network)[network]

elmo_4x4096_512_4                   0.370091
elmo_original_2                     0.384753
openai_transformer_12               0.388729
calypso_transformer_6_512_base_6    0.396797
bert_large_cased_24                 0.406230
openai_transformer_11               0.407604
elmo_4x4096_512_3                   0.415548
openai_transformer_10               0.422016
bert_large_cased_23                 0.427605
calypso_transformer_6_512_base_5    0.428445
openai_transformer_9                0.431988
bert_base_cased_12                  0.432118
bert_large_cased_22                 0.434581
bert_large_cased_21                 0.439737
openai_transformer_8                0.442072
bert_large_cased_20                 0.444810
bert_base_cased_11                  0.448078
bert_large_cased_19                 0.449919
elmo_4x4096_512_2                   0.451982
bert_large_cased_18                 0.457278
openai_transformer_7                0.459244
bert_base_cased_10                  0.459536
calypso_tr